In [ ]:
import random
from combination_generator import *
from IPython.display import Image, SVG, HTML

class Player:
  def __init__(self, name):
    self.cards = []
    self.name = name

  def give_card(self, card):
    self.cards.append(card)
  
  def play_hand(self, pile):
    valid_combination = []
    combinations = combination_from_pile(self.cards)
    for c in combinations:
      if compare_combinations(pile, c) < 0:
        valid_combination.append(c)
    if valid_combination == []:
      return None

    random.shuffle(valid_combination) # pick a move randomly  
    return valid_combination[0]

  def first_play(self):
    valid_combinations = []
    combinations = combination_from_pile(self.cards)
    for c in combinations:
      _, cards = c
      if 0 in cards:
        valid_combinations.append(c)
    random.shuffle(valid_combinations)
    return valid_combinations[0]


def simulate_game(number_of_players = 4):
  deck = []
  pile = []
  for x in range(52):
    deck.append(x)

  random.shuffle(deck)
  print(deck)
  
  players = [Player(f"{p + 1}") for p in range(number_of_players)]

  def describe(cards):
    card_images = []
    for c in cards:
      _, suite, card, index = card_id_to_name(c)
      suite_id = index // 13
      if suite_id == 0:
        suite_id = 2
      elif suite_id == 1:
        suite_id = 0
      elif suite_id == 2:
        suite_id = 1
      card_images.append(f"faces/{suite_id}_{((index + 2) % 13) + 1}.svg")
    imagesList=''.join( ["<img style='width: 120px; margin: 0px; float: left; border: 1px solid black; background: white;' src='%s' />" % str(s) 
                for s in sorted(card_images) ])
    display(HTML(imagesList))


  def win_condition(num_of_players):
    total = 0
    for p in players:
      if len(p.cards) == 0:
        total += 1
    if total == num_of_players - 1:
      return True
    return False

  for x in range(13):
    for p in players:
      p.give_card(deck.pop())
  
  # print hands
  print(f"starting hands:")
  for p_num in range(len(players)):
    p = players[p_num]
    print(f"For Player {p.name}:")
    describe(p.cards)
    print(f"")
    if 0 in p.cards and p_num != 0:
      print(f"player {p.name} has 3 clubs")
      p_temp = players[0]
      players[0] = players[p_num]
      players[p_num] = p_temp

  current_turn = 0
  last_play = None
  last_move = None

  print(f"Game start!")
  print("-------------")

  while not win_condition(number_of_players):
    current_player = players[current_turn % number_of_players]
    print(" ")
    print(f"turn {current_turn} =============================")
    print(f"player {current_player.name} cards in hand {len(current_player.cards)}:")
    if last_move == None and last_play == current_player:
      pile = []
      
    if current_turn == 0:
      move = current_player.first_play()
    else:
      move = current_player.play_hand(pile)
      
    if move != None:
      pile = move
      last_play = current_player
      last_move = move
      _, cards = move
      describe(cards)
      for c in cards:
        current_player.cards.remove(c)
    else:
      print('pass')
      last_move = None
    current_turn += 1
  for p in players:
    if len(p.cards) > 0:
      print(f"Player #{p.name} loses!")
    

In [ ]:

simulate_game()


In [ ]:

# pile = []
# for x in range(52):
#   pile.append(x)
  
# result = combination_from_pile(pile)


# total_combinations = len(result)

# ratios = []
# print(f"total combinations = {total_combinations}")
# for type in range(9):
#   count = len([x for x in filter(lambda x: x[0] == type, result)])
#   print(f"{type}: count = {count} / {total_combinations}")
#   ratio = count / total_combinations
#   ratios.append([type, ratio])
# print(ratios)

# for type, cards in result:
#   print(f"comb type {type}")
#   for c in cards:
#     _, suite, card, index = card_id_to_name(c)
#     print(f"{suite} {card}")